# Video compression

According to the dataset paper, 

> However, for each target video in the test set, we randomly selected two clips out of three and applied augmentations that
approximate actual degradations seen in real-life video distributions. Specifically, these augmentations were
> 1. reduce the FPS of the video to 15
> 2. reduce the resolution of the video to 1/4 of its original size
> 3. reduce the overall encoding quality

In this notebook we look at how to compress videos using `ffmpeg`.  Based on the above list, a quick search returns several useful examples:

* General compression for the web 1: https://dev.to/benjaminblack/use-ffmpeg-to-compress-and-convert-videos-458l  
* General compression for the web 2: https://trac.ffmpeg.org/wiki/Encode/YouTube   
* Frame rate change: https://trac.ffmpeg.org/wiki/ChangingFrameRate  
* Scaling: https://trac.ffmpeg.org/wiki/Scaling  

The latest `ffmpeg` can be obtained at https://johnvansickle.com/ffmpeg/.  See the FAQ for how to install.

In [ ]:
from IPython.display import HTML
import cv2
from fastai.core import *
from fastai.vision import *
from kgl_deepfake.data import *

In [ ]:
SOURCE = Path('../data/dfdc_train_part_0/')

In [ ]:
fnames = get_files(SOURCE, extensions=['.mp4'])

In [ ]:
fn = fnames[159]
fn

PosixPath('../data/dfdc_train_part_0/bfktkdugru.mp4')

In [ ]:
HTML(html_vid(fn))

Use `mediainfo` to show video properties

In [ ]:
! mediainfo {fn}

General
Complete name                            : ../data/dfdc_train_part_0/bfktkdugru.mp4
Format                                   : MPEG-4
Format profile                           : Base Media
Codec ID                                 : isom (isom/iso2/avc1/mp41)
File size                                : 15.2 MiB
Duration                                 : 10 s 24 ms
Overall bit rate                         : 12.7 Mb/s
Writing application                      : Lavf57.71.100

Video
ID                                       : 1
Format                                   : AVC
Format/Info                              : Advanced Video Codec
Format profile                           : High@L4
Format settings                          : CABAC / 4 Ref Frames
Format settings, CABAC                   : Yes
Format settings, Reference frames        : 4 frames
Codec ID                                 : avc1
Codec ID/Info                            : Advanced Video Coding
Duration                    

### File size

To conveniently check the file size of compressed videos:

In [ ]:
def get_file_size(fpath):
    assert fpath.is_file()
    p = subprocess.run(f"du -hs {fpath}".split(), stdout=subprocess.PIPE)
    return p.stdout.decode().split('\t')[0]

In [ ]:
Path.file_size = get_file_size

Original file size

In [ ]:
fn.file_size()

' 15M'

### Decorator to run command in terminal

In [ ]:
def runnit(f):
    def _func(*args, **kwargs):
        command = f(*args, **kwargs)
        p = subprocess.run(command.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return p
    return _func

### Are these videos HDR?

The following checks whether the video is HDR or not.

In [ ]:
@runnit
def vid_colour_primaries(fpath):
    "Use `mediainfo` to get video's colour primaries."
    return f'''
    mediainfo f'{fpath}' --Inform="Video;%colour_primaries%"
    '''

def vid_is_HDR(fpath):
    "Is video HDR or not?"
    p = vid_colour_primaries(fpath)
    res = p.stdout.decode()
    return True if 'BT.2020' in res else False

In [ ]:
vid_is_HDR(fn)

False

### Show video properties using `mediainfo`

In [ ]:
def get_mediainfo(fpath=None): 
    @runnit
    def _func(fpath=None):
        return f'''mediainfo {fpath}'''
    p = _func(fpath=fpath)
    return p.stdout.decode()

### Encoding

Each of the URLs above does something different: encoding, scaling, fps.  These are combined into a single function.  Note that probably not all combinations of input parameters will run, but the main parameters to adjust are:

* **crf** "The range of the CRF scale is 0–51, where 0 is lossless, 23 is the default, and 51 is worst quality possible. A lower value generally leads to higher quality, and a subjectively sane range is 17–28. Consider 17 or 18 to be visually lossless or nearly so; it should look the same or nearly the same as the input but it isn't technically lossless." > 23 recommended.
* **fps** "Changing frame rates requires the video to be re-encoded. Without setting appropriate output quality or bit rate, the video quality may be degraded. Please look at the respective encoding guides for the codec you've chosen."
* **scale** Value between 0 and 1 to scale the original lengths.

To display the video and its properties after it's been compressed

In [ ]:
def show_vid_info(f):
    "Display video and its properties after it's been processed."
    def _f(*args, **kwargs):
        p = f(*args, **kwargs)
        fpath = Path(kwargs['fpath_to'])
        display(HTML(html_vid(fpath)))
        print(get_mediainfo(fpath))
    return _f

Function for donig the encoding using `ffmpeg`, with default arguments defined.

In [ ]:
def _ffmpeg_web_defaults():
    '''
    These are some reasonable values for uploading. i.e. YouTube, etc.
    '''
    return dict(video_encoder='libx264', video_bitrate='1.5M', fps=30, scale=.5, crf=23, #17-28
                audio_encoder='aac', audio_bitrate='128k')

def _ffmpeg_defaults():
    return dict(video_encoder=None, video_bitrate=None, fps=None, scale=None, crf=None,
                audio_encoder=None, audio_bitrate=None)

def _ffmpeg_fmts():
    "ffmpeg options syntax"
    return dict(video_encoder='-c:v {video_encoder:s}', 
                video_bitrate='-b:v {video_bitrate:s}', 
                fps='-r {fps:d}', 
                scale='-vf scale=iw*{scale:.2f}:ih*{scale:.2f}', 
                crf='-crf {crf:d}',
                audio_bitrate='-b:a {audio_bitrate}', 
                audio_encoder='-c:a {audio_encoder}')

@show_vid_info
@runnit
def run_ffmpeg(fpath_from=None, fpath_to=None, **kwargs):
    '''
    Run ffmpeg
    '''
    ps = _ffmpeg_defaults()
    ps.update(kwargs)
    pstr = []
    for n, s in _ffmpeg_fmts().items():
        if ps[n] is None: pstr.append('')
        else:
            pstr.append(s.format(**ps))
    pstr = ' '.join(pstr)
    print(f'ffmpeg -i {fpath_from} {pstr} {fpath_to}')
    return f'ffmpeg -i {fpath_from} {pstr} {fpath_to}'

Apply to the original:

In [ ]:
f = Path(f'enc_{fn.name}')
if f.exists(): os.remove(f)
run_ffmpeg(fpath_from=fn, fpath_to=f, crf=60, scale=.25, fps=15)

ffmpeg -i ../data/dfdc_train_part_0/bfktkdugru.mp4   -r 15 -vf scale=iw*0.25:ih*0.25 -crf 60   enc_bfktkdugru.mp4


General
Complete name                            : enc_bfktkdugru.mp4
Format                                   : MPEG-4
Format profile                           : Base Media
Codec ID                                 : isom (isom/iso2/avc1/mp41)
File size                                : 101 KiB
Duration                                 : 10 s 134 ms
Overall bit rate                         : 81.6 kb/s
Writing application                      : Lavf58.29.100

Video
ID                                       : 1
Format                                   : AVC
Format/Info                              : Advanced Video Codec
Format profile                           : High@L2.1
Format settings                          : CABAC / 4 Ref Frames
Format settings, CABAC                   : Yes
Format settings, Reference frames        : 4 frames
Codec ID                                 : avc1
Codec ID/Info                            : Advanced Video Coding
Duration                                 : 10 s 

Double check the properties using `cv2`

In [ ]:
vcap = cv2.VideoCapture(str(f))
vlen = int(vcap.get(cv2.CAP_PROP_FRAME_COUNT))
ret, frame = vcap.read()
vcap.release()
vlen, frame.shape

(152, (480, 270, 3))

# -fin